In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
import tensorflow as tf

In [ ]:
folder_benign_train = '../input/skin-cancer-malignant-vs-benign/train/benign'
folder_malignant_train = '../input/skin-cancer-malignant-vs-benign/train/malignant'

folder_benign_test = '../input/skin-cancer-malignant-vs-benign/test/benign'
folder_malignant_test = '../input/skin-cancer-malignant-vs-benign/test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

# Load in training pictures 
ims_benign = [read(os.path.join(folder_benign_train, filename)) for filename in os.listdir(folder_benign_train)]
X_benign = np.array(ims_benign, dtype='uint8')
ims_malignant = [read(os.path.join(folder_malignant_train, filename)) for filename in os.listdir(folder_malignant_train)]
X_malignant = np.array(ims_malignant, dtype='uint8')

# Load in testing pictures
ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
X_benign_test = np.array(ims_benign, dtype='uint8')
ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
X_malignant_test = np.array(ims_malignant, dtype='uint8')

# Create labels
y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])

y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])


# Merge data 
X_train = np.concatenate((X_benign, X_malignant), axis = 0)
y_train = np.concatenate((y_benign, y_malignant), axis = 0)

X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis = 0)

# Shuffle data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(224, 224,3)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
  )

In [ ]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 50)

In [ ]:
import PIL
from PIL import Image
def convertImage(dir):
    load_img_rz = np.array(Image.open(dir).resize((224,224)))
    load_img_rz = load_img_rz.reshape(-3, 224, 224, 3)
    return load_img_rz

In [ ]:
def predictionAnalysis(dir):
    convertedimage = convertImage(dir)
    results = model.predict(convertedimage)
    benignchance = results[0][0] * 100
    malignantchance = results[0][1] * 100
    print(str(benignchance)+'% chance of benign and '+str(malignantchance)+'% chance of malignant')
                                  

In [ ]:
predictionAnalysis('../input/melanoma-picture-malignant/1200px-Melanoma.jpg')

In [ ]:
predictionAnalysis('../input/skin-cancer-benign/Skin Cancer Benign.jpg')

In [ ]:
predictionAnalysis('../input/basal-skin-carcinoma/basal_cell_carcinoma_b_high.jpg')

In [ ]:
predictionAnalysis('../input/basal-skin-carcinoma/what-is-basal-cell-carcinoma-skin-cancer-1440x810.jpg')

In [ ]:
predictionAnalysis('../input/spitz-nevus/300px-Spitz_nevus.jpg')

In [ ]:
predictionAnalysis('../input/spitz-nevus-2/800wm.jpg')